# P2P Algorithm Labs

In this notebook you will explore the four algorithms that make a P2P network work:

| Lab | Algorithm | What It Solves |
|-----|-----------|----------------|
| 1 | **Gossip Protocol** | How do nodes discover each other without a central registry? |
| 2 | **Heartbeat** | How do you know if a peer is alive, slow, or dead? |
| 3 | **Choking** | Why should a peer cooperate? What happens to free-riders? |
| 4 | **Reputation** | A peer contributes but lies about view counts. How do you detect that? |

**No AWS needed.** Everything here is a local simulation. You will tweak parameters,
observe how behavior changes, and build intuition for how these algorithms behave
at scale.

Each lab ends with questions. Answer them in your own words -- understanding *why*
matters more than getting the "right" answer.

---
# Lab 1: Gossip Protocol

## The Problem

You have 8 nodes. No central registry. Each node starts knowing only *one* neighbor.
How does everyone find out about everyone else?

## The Algorithm

Each round, every node:
1. Picks a **random** peer from its known list
2. Sends its entire peer list to that peer
3. Merges any new peers it learns about

New peers spread through the network like a rumor -- hence the name "gossip."

**Key question:** How many rounds until all 8 nodes know about all 8?

### Setup: The Gossip Classes

Run the next cell to define the gossip simulation.
Read the code -- it is short and each method does one thing.

In [ ]:
# ============================
# GOSSIP PROTOCOL SIMULATION
# ============================

import random
from dataclasses import dataclass, field

@dataclass
class PeerEntry:
    """A single entry in a node's peer list."""
    node_id: str
    queue_url: str
    last_seen: float = 0.0
    ttl: int = 5

    def is_expired(self):
        return self.ttl <= 0


class GossipNode:
    """A node that participates in gossip protocol."""

    def __init__(self, node_id, queue_url=""):
        self.node_id = node_id
        self.queue_url = queue_url or f"sqs://fake/{node_id}"
        self.peers = {}  # node_id -> PeerEntry

    def add_peer(self, node_id, queue_url=""):
        """Add a peer manually (bootstrap)."""
        if node_id != self.node_id and node_id not in self.peers:
            self.peers[node_id] = PeerEntry(
                node_id=node_id,
                queue_url=queue_url or f"sqs://fake/{node_id}",
            )

    def get_peer_list_message(self):
        """Build a PEER_LIST message payload."""
        entries = [{"node_id": p.node_id, "queue_url": p.queue_url} for p in self.peers.values()]
        entries.append({"node_id": self.node_id, "queue_url": self.queue_url})
        return entries

    def receive_peer_list(self, incoming, sender_id):
        """Merge an incoming peer list with our own."""
        new_count = 0
        for entry in incoming:
            nid = entry["node_id"]
            if nid == self.node_id:
                continue
            if nid not in self.peers:
                self.peers[nid] = PeerEntry(
                    node_id=nid,
                    queue_url=entry.get("queue_url", ""),
                )
                new_count += 1
            else:
                self.peers[nid].ttl = 5  # refresh
        return new_count

    def age_entries(self):
        """Decrement TTL on all entries. Remove expired ones."""
        expired = []
        for nid, entry in self.peers.items():
            entry.ttl -= 1
            if entry.is_expired():
                expired.append(nid)
        for nid in expired:
            del self.peers[nid]
        return expired

    def pick_gossip_target(self):
        if not self.peers:
            return None
        return random.choice(list(self.peers.keys()))

    def known_peer_count(self):
        return len(self.peers)


print("GossipNode defined. Ready for simulation.")

### 1A: Basic convergence

8 nodes in a ring -- each knows only its clockwise neighbor.
Watch how gossip spreads knowledge across the network.

In [ ]:
# 1A: Ring topology - how many rounds to converge?

random.seed(42)  # reproducible

node_ids = [f"node-{i}" for i in range(8)]
nodes = {nid: GossipNode(nid) for nid in node_ids}

# Ring: each node knows only its next neighbor
for i, nid in enumerate(node_ids):
    next_id = node_ids[(i + 1) % len(node_ids)]
    nodes[nid].add_peer(next_id)

print("Initial knowledge (ring):")
for nid in node_ids:
    known = list(nodes[nid].peers.keys())
    print(f"  {nid} knows: {known}")

# Run gossip rounds
convergence_round = None
for rnd in range(1, 16):
    # Each node picks a random peer and exchanges lists
    for nid in node_ids:
        target_id = nodes[nid].pick_gossip_target()
        if target_id:
            msg = nodes[nid].get_peer_list_message()
            new = nodes[target_id].receive_peer_list(msg, nid)
            # Target also shares back
            reply = nodes[target_id].get_peer_list_message()
            nodes[nid].receive_peer_list(reply, target_id)

    # Check convergence
    counts = [nodes[nid].known_peer_count() for nid in node_ids]
    min_known = min(counts)
    max_known = max(counts)
    full = all(c == len(node_ids) - 1 for c in counts)

    print(f"  Round {rnd:2d}: min={min_known}, max={max_known}, full={'YES' if full else 'no'}")

    if full and convergence_round is None:
        convergence_round = rnd

print(f"\nConverged at round: {convergence_round}")
print(f"Theoretical bound: O(log {len(node_ids)}) = ~{len(node_ids).bit_length()} rounds")

**Observation:** Gossip converges in roughly O(log N) rounds. For 8 nodes,
that is about 3 rounds. The random peer selection means information spreads
exponentially -- each round roughly doubles the reach.

> **Question 1:** What happens if you increase the network to 16 nodes? 32?
> Does the number of rounds roughly double, or roughly increase by 1?

### 1B: What if you change the seed?

The random seed affects which peers get chosen. Run with different seeds
to see how much variance there is.

In [ ]:
# 1B: Run 10 trials with different seeds

results = []
for seed in range(10):
    random.seed(seed)
    nodes = {nid: GossipNode(nid) for nid in node_ids}
    for i, nid in enumerate(node_ids):
        nodes[nid].add_peer(node_ids[(i + 1) % len(node_ids)])

    for rnd in range(1, 20):
        for nid in node_ids:
            target_id = nodes[nid].pick_gossip_target()
            if target_id:
                msg = nodes[nid].get_peer_list_message()
                nodes[target_id].receive_peer_list(msg, nid)
                reply = nodes[target_id].get_peer_list_message()
                nodes[nid].receive_peer_list(reply, target_id)

        if all(nodes[nid].known_peer_count() == len(node_ids) - 1 for nid in node_ids):
            results.append(rnd)
            break
    else:
        results.append(None)

print("Convergence rounds across 10 seeds:")
for i, r in enumerate(results):
    bar = "#" * (r or 0)
    print(f"  Seed {i}: {r:2d} rounds  {bar}")
print(f"\nAverage: {sum(r for r in results if r) / len([r for r in results if r]):.1f} rounds")

### 1C: Scaling experiment

How does convergence scale with network size?

In [ ]:
# 1C: Convergence vs network size

import math

print(f"{'Nodes':>6}  {'Avg Rounds':>10}  {'O(log N)':>8}  {'Bar'}")
print("-" * 45)

for n in [4, 8, 16, 32, 64]:
    trials = []
    for seed in range(5):
        random.seed(seed)
        ids = [f"n{i}" for i in range(n)]
        ns = {nid: GossipNode(nid) for nid in ids}
        for i, nid in enumerate(ids):
            ns[nid].add_peer(ids[(i + 1) % n])

        for rnd in range(1, 50):
            for nid in ids:
                t = ns[nid].pick_gossip_target()
                if t:
                    msg = ns[nid].get_peer_list_message()
                    ns[t].receive_peer_list(msg, nid)
                    reply = ns[t].get_peer_list_message()
                    ns[nid].receive_peer_list(reply, t)
            if all(ns[nid].known_peer_count() == n - 1 for nid in ids):
                trials.append(rnd)
                break

    avg = sum(trials) / len(trials) if trials else 0
    log_n = math.log2(n)
    bar = "#" * int(avg)
    print(f"{n:6d}  {avg:10.1f}  {log_n:8.1f}  {bar}")

> **Question 2:** The convergence rounds grow roughly logarithmically.
> Why is this? Think about what happens each round -- how many nodes
> learn about a new peer?

> **Question 3:** In our 2032 system with 5 students + 3 bots + 1 instructor
> (9 nodes), how many gossip rounds would you expect?

---
# Lab 2: Heartbeat / Liveness Detection

## The Problem

You know 5 peers. One crashes at round 5. One is slow (responds only 50% of the time).
How quickly can you detect the crash without false-alarming on the slow peer?

## The Algorithm

Every round:
1. Send PING to all known peers
2. Wait for PONG responses
3. Track consecutive misses per peer
4. State transitions: ALIVE -> SUSPECT -> DEAD

**Key tension:** Aggressive thresholds detect failures fast but cause false positives.
Conservative thresholds are safe but slow.

In [ ]:
# ============================
# HEARTBEAT SIMULATION
# ============================

from enum import Enum
from dataclasses import dataclass
from typing import Dict, List, Optional

class PeerStatus(str, Enum):
    ALIVE = "ALIVE"
    SUSPECT = "SUSPECT"
    DEAD = "DEAD"


@dataclass
class PeerState:
    """Tracked state for a known peer."""
    node_id: str
    status: PeerStatus = PeerStatus.ALIVE
    consecutive_misses: int = 0
    last_pong_round: int = 0
    total_pings_sent: int = 0
    total_pongs_received: int = 0

    def response_rate(self):
        if self.total_pings_sent == 0:
            return 1.0
        return self.total_pongs_received / self.total_pings_sent


class HeartbeatNode:
    """A node that monitors peer liveness."""

    def __init__(self, node_id, miss_threshold=3, grace_period=2):
        self.node_id = node_id
        self.miss_threshold = miss_threshold
        self.grace_period = grace_period
        self.peers = {}
        self.log = []

    def add_peer(self, node_id):
        self.peers[node_id] = PeerState(node_id=node_id)

    def send_pings(self, current_round):
        pinged = []
        for nid, state in self.peers.items():
            if state.status != PeerStatus.DEAD:
                state.total_pings_sent += 1
                pinged.append(nid)
        return pinged

    def receive_pong(self, from_node, current_round):
        if from_node in self.peers:
            state = self.peers[from_node]
            state.total_pongs_received += 1
            state.last_pong_round = current_round
            old = state.status
            state.consecutive_misses = 0
            state.status = PeerStatus.ALIVE
            if old != PeerStatus.ALIVE:
                self.log.append(f"  Round {current_round}: {from_node} recovered ({old} -> ALIVE)")

    def record_miss(self, peer_id, current_round):
        if peer_id not in self.peers:
            return
        state = self.peers[peer_id]
        if state.status == PeerStatus.DEAD:
            return
        state.consecutive_misses += 1
        old = state.status
        if state.consecutive_misses >= self.miss_threshold:
            state.status = PeerStatus.DEAD
        elif state.consecutive_misses >= self.grace_period:
            state.status = PeerStatus.SUSPECT
        if state.status != old:
            self.log.append(f"  Round {current_round}: {peer_id} {old} -> {state.status} (misses={state.consecutive_misses})")

    def get_alive(self):
        return [nid for nid, s in self.peers.items() if s.status == PeerStatus.ALIVE]

    def get_dead(self):
        return [nid for nid, s in self.peers.items() if s.status == PeerStatus.DEAD]

    def get_suspect(self):
        return [nid for nid, s in self.peers.items() if s.status == PeerStatus.SUSPECT]


print("HeartbeatNode defined. Ready for simulation.")

### 2A: Aggressive vs Conservative thresholds

We will run the same scenario twice:
- **Aggressive:** threshold=2 (declare dead after 2 missed PINGs)
- **Conservative:** threshold=5 (wait 5 misses)

The scenario: 5 peers. `peer-C` crashes at round 5. `peer-D` is slow (50% responses).

In [ ]:
# 2A: Compare aggressive vs conservative

random.seed(42)

def run_heartbeat_sim(threshold, grace, label):
    print(f"\n{'='*50}")
    print(f"  {label}: threshold={threshold}, grace={grace}")
    print(f"{'='*50}")

    node = HeartbeatNode("monitor", miss_threshold=threshold, grace_period=grace)
    peers = ["peer-A", "peer-B", "peer-C", "peer-D", "peer-E"]
    for p in peers:
        node.add_peer(p)

    for rnd in range(1, 16):
        node.send_pings(rnd)

        for p in peers:
            if p == "peer-C" and rnd >= 5:
                # peer-C crashed at round 5
                node.record_miss(p, rnd)
            elif p == "peer-D" and random.random() > 0.5:
                # peer-D responds 50% of the time
                node.record_miss(p, rnd)
            else:
                node.receive_pong(p, rnd)

    # Print log
    for entry in node.log:
        print(entry)

    # Summary
    print(f"\n  Final states:")
    for nid, state in node.peers.items():
        print(f"    {nid}: {state.status.value} (response rate: {state.response_rate():.0%})")

    false_positives = [nid for nid in ["peer-A", "peer-B", "peer-D", "peer-E"]
                       if node.peers[nid].status == PeerStatus.DEAD]
    if false_positives:
        print(f"\n  ** FALSE POSITIVES: {false_positives} were declared DEAD but are not crashed **")

    if node.peers["peer-C"].status == PeerStatus.DEAD:
        # Find the round it was declared dead
        for entry in node.log:
            if "peer-C" in entry and "DEAD" in entry:
                print(f"  ** Detection time: {entry.strip()} **")
                break

    return node


# Run both
random.seed(42)
aggressive = run_heartbeat_sim(threshold=2, grace=1, label="AGGRESSIVE")
random.seed(42)
conservative = run_heartbeat_sim(threshold=5, grace=3, label="CONSERVATIVE")

> **Question 4:** Which threshold correctly detected peer-C's crash
> without false-alarming on peer-D?

> **Question 5:** In a real P2P ad counting system, which is worse:
> - Declaring a live peer dead (they get excluded from votes), or
> - Keeping a dead peer in your list (wasting time pinging them)?
>
> How does this inform your choice of threshold?

### 2B: The grace period effect

The SUSPECT state gives slow peers a chance to recover before being declared DEAD.
Let's see how different grace periods affect false positive rates.

In [ ]:
# 2B: Measure false positive rates across thresholds

print(f"{'Threshold':>10} {'Grace':>6} {'Detected crash?':>16} {'False positives':>16}")
print("-" * 55)

for threshold in [2, 3, 4, 5]:
    for grace in range(1, threshold):
        random.seed(42)
        node = HeartbeatNode("monitor", miss_threshold=threshold, grace_period=grace)
        peers = ["peer-A", "peer-B", "peer-C", "peer-D", "peer-E"]
        for p in peers:
            node.add_peer(p)

        for rnd in range(1, 16):
            node.send_pings(rnd)
            for p in peers:
                if p == "peer-C" and rnd >= 5:
                    node.record_miss(p, rnd)
                elif p == "peer-D" and random.random() > 0.5:
                    node.record_miss(p, rnd)
                else:
                    node.receive_pong(p, rnd)

        detected = node.peers["peer-C"].status == PeerStatus.DEAD
        fps = sum(1 for nid in ["peer-A", "peer-B", "peer-D", "peer-E"]
                  if node.peers[nid].status == PeerStatus.DEAD)
        print(f"{threshold:>10} {grace:>6} {'YES' if detected else 'no':>16} {fps:>16}")

> **Question 6:** What is the smallest threshold/grace combination that
> detects peer-C's crash with zero false positives? Why?

---
# Lab 3: Choking / Unchoking (Reciprocity)

## The Problem

You have 6 peers. Most are cooperative -- they share data with you. But two are
*free-riders*: they consume your data without contributing anything back.

How do you reward cooperators and punish free-riders?

## The Algorithm (from BitTorrent)

Every round:
1. Rank peers by how much they contributed to you
2. **Unchoke** the top N contributors (serve them)
3. **Choke** everyone else (cut them off)
4. **Optimistic unchoke:** Every K rounds, randomly unchoke one choked peer
   (gives new/cold-start peers a chance to prove themselves)

In [ ]:
# ============================
# CHOKING SIMULATION
# ============================

@dataclass
class PeerTracker:
    """Tracks a peer's contribution and choking state."""
    node_id: str
    contributed: int = 0        # what they gave us
    received: int = 0           # what we gave them
    is_choked: bool = True      # are we choking them?
    is_interested: bool = True  # do they want our data?
    rounds_choked: int = 0

    def reciprocity_ratio(self):
        if self.received == 0:
            return float('inf') if self.contributed > 0 else 0.0
        return self.contributed / self.received


class ChokingNode:
    """A node that implements BitTorrent-style choking."""

    def __init__(self, node_id, max_unchoked=4, optimistic_interval=3):
        self.node_id = node_id
        self.max_unchoked = max_unchoked
        self.optimistic_interval = optimistic_interval
        self.peers = {}
        self.round_count = 0
        self.optimistic_peer = None
        self.log = []

    def add_peer(self, node_id, interested=True):
        self.peers[node_id] = PeerTracker(node_id=node_id, is_interested=interested)

    def record_contribution(self, from_peer, units=1):
        if from_peer in self.peers:
            self.peers[from_peer].contributed += units

    def run_choking_round(self):
        self.round_count += 1
        interested = {nid: p for nid, p in self.peers.items() if p.is_interested}
        if not interested:
            return

        # Sort by contribution (descending)
        ranked = sorted(interested.values(), key=lambda p: p.contributed, reverse=True)

        # Top N-1 get unchoked (reserve 1 slot for optimistic)
        regular_slots = self.max_unchoked - 1
        to_unchoke = set()
        for p in ranked[:regular_slots]:
            to_unchoke.add(p.node_id)

        # Optimistic unchoke
        if self.round_count % self.optimistic_interval == 0:
            choked_interested = [p for p in ranked if p.node_id not in to_unchoke]
            if choked_interested:
                self.optimistic_peer = random.choice(choked_interested).node_id
                self.log.append(f"  Round {self.round_count}: Optimistic unchoke -> {self.optimistic_peer}")

        if self.optimistic_peer:
            to_unchoke.add(self.optimistic_peer)

        # Apply decisions
        for nid, p in interested.items():
            old_choked = p.is_choked
            p.is_choked = nid not in to_unchoke
            if p.is_choked:
                p.rounds_choked += 1
            if old_choked and not p.is_choked:
                self.log.append(f"  Round {self.round_count}: UNCHOKED {nid} (contributed={p.contributed})")
            elif not old_choked and p.is_choked:
                self.log.append(f"  Round {self.round_count}: CHOKED {nid} (contributed={p.contributed})")


print("ChokingNode defined. Ready for simulation.")

### 3A: Free-riders get choked

6 peers: 4 cooperate (contribute 2-5 units/round), 2 are free-riders (contribute 0).
Watch what happens over 15 rounds.

In [ ]:
# 3A: Free-riders vs cooperators

random.seed(42)

node = ChokingNode("monitor", max_unchoked=4, optimistic_interval=3)

peers = {
    "alice":   {"generous": 5},  # very generous
    "bob":     {"generous": 3},
    "carol":   {"generous": 2},
    "dave":    {"generous": 4},
    "freeloader-1": {"generous": 0},
    "freeloader-2": {"generous": 0},
}

for nid in peers:
    node.add_peer(nid)

print("=== Choking Simulation: 15 rounds ===\n")

for rnd in range(1, 16):
    # Peers contribute
    for nid, cfg in peers.items():
        if cfg["generous"] > 0:
            amount = random.randint(1, cfg["generous"])
            node.record_contribution(nid, amount)

    # Choking round
    node.run_choking_round()

# Print log
for entry in node.log:
    print(entry)

# Final state
print(f"\nFinal standings after 15 rounds:\n")
print(f"  {'Peer':<16} {'Contributed':>12} {'Status':>10} {'Rounds Choked':>14}")
print(f"  {'-'*54}")
for nid in sorted(node.peers, key=lambda x: node.peers[x].contributed, reverse=True):
    p = node.peers[nid]
    status = "CHOKED" if p.is_choked else "UNCHOKED"
    print(f"  {nid:<16} {p.contributed:>12} {status:>10} {p.rounds_choked:>14}")

> **Question 7:** Did the free-riders end up choked? How many rounds
> did it take before they were consistently cut off?

> **Question 8:** Without optimistic unchoking, a new peer who has never
> contributed would *always* be choked. How does optimistic unchoking solve
> the cold-start problem?

### 3B: What happens when you change max_unchoked?

With `max_unchoked=4`, only the top 4 contributors get served.
What if you increase or decrease that number?

In [ ]:
# 3B: Effect of max_unchoked parameter

print(f"{'max_unchoked':>13} {'Cooperators served':>20} {'Free-riders served':>20}")
print("-" * 55)

for max_u in [2, 3, 4, 5, 6]:
    random.seed(42)
    node = ChokingNode("monitor", max_unchoked=max_u, optimistic_interval=3)
    for nid in peers:
        node.add_peer(nid)

    for rnd in range(1, 16):
        for nid, cfg in peers.items():
            if cfg["generous"] > 0:
                node.record_contribution(nid, random.randint(1, cfg["generous"]))
        node.run_choking_round()

    coop_unchoked = sum(1 for nid in ["alice","bob","carol","dave"] if not node.peers[nid].is_choked)
    free_unchoked = sum(1 for nid in ["freeloader-1","freeloader-2"] if not node.peers[nid].is_choked)
    print(f"{max_u:>13} {coop_unchoked:>20} {free_unchoked:>20}")

print(f"\nWith max_unchoked=6 all peers are served -- choking is disabled.")
print(f"With max_unchoked=2 even some cooperators get cut off.")

> **Question 9:** For the 2032 ad counting system (9 nodes), what would you
> set `max_unchoked` to? Can you set it too low? Too high?

---
# Lab 4: Reputation Scoring

## The Problem

Choking handles free-riders -- peers who do not contribute. But what about
peers who contribute *wrong data*? A peer could respond to every PING,
contribute data, and still *lie* about view counts to inflate revenue.

Choking cannot detect this. You need a trust system.

## The Algorithm

Track three metrics per peer:
- **Accuracy:** What percentage of their reports match the majority?
- **Uptime:** What percentage of heartbeats do they respond to?
- **Reciprocity:** Do they contribute data or just consume?

Combine these into a **trust score** (0.0 to 1.0), then use trust scores
as voting weights when auditing.

In [ ]:
# ============================
# REPUTATION SIMULATION
# ============================

@dataclass
class ReputationRecord:
    """Tracks reputation metrics for a single peer."""
    node_id: str
    reports_total: int = 0
    reports_accurate: int = 0
    heartbeats_total: int = 0
    heartbeats_responded: int = 0
    contributions: int = 0
    consumptions: int = 0
    _trust_score: float = 0.5

    def accuracy(self):
        if self.reports_total == 0:
            return 0.5  # uncertain
        return self.reports_accurate / self.reports_total

    def uptime(self):
        if self.heartbeats_total == 0:
            return 0.5
        return self.heartbeats_responded / self.heartbeats_total

    def reciprocity(self):
        total = self.contributions + self.consumptions
        if total == 0:
            return 0.5
        return self.contributions / total

    @property
    def trust_score(self):
        return self._trust_score

    def recalculate_trust(self, weights=(0.5, 0.3, 0.2)):
        """Weighted combination of accuracy, uptime, reciprocity."""
        w_acc, w_up, w_rec = weights
        raw = (w_acc * self.accuracy() + w_up * self.uptime() + w_rec * self.reciprocity())
        # Decay toward neutral for low data
        data_points = self.reports_total + self.heartbeats_total
        confidence = min(1.0, data_points / 20)
        self._trust_score = confidence * raw + (1 - confidence) * 0.5


class ReputationNode:
    """A node that tracks reputation of its peers."""

    def __init__(self, node_id):
        self.node_id = node_id
        self.peers = {}

    def add_peer(self, node_id):
        self.peers[node_id] = ReputationRecord(node_id=node_id)

    def record_report(self, peer_id, was_accurate):
        if peer_id in self.peers:
            p = self.peers[peer_id]
            p.reports_total += 1
            if was_accurate:
                p.reports_accurate += 1

    def record_heartbeat(self, peer_id, responded):
        if peer_id in self.peers:
            p = self.peers[peer_id]
            p.heartbeats_total += 1
            if responded:
                p.heartbeats_responded += 1

    def record_contribution(self, peer_id, units=1):
        if peer_id in self.peers:
            self.peers[peer_id].contributions += units

    def record_consumption(self, peer_id, units=1):
        if peer_id in self.peers:
            self.peers[peer_id].consumptions += units

    def update_all_scores(self, weights=(0.5, 0.3, 0.2)):
        for rec in self.peers.values():
            rec.recalculate_trust(weights)

    def weighted_majority_vote(self, votes, verbose=False):
        """Reputation-weighted majority vote.

        Args:
            votes: {peer_id: their_reported_count}
        Returns:
            (winning_value, confidence)
        """
        if not votes:
            return None, 0.0

        # Accumulate weighted votes per value
        weighted = {}
        total_weight = 0
        for peer_id, value in votes.items():
            weight = self.peers[peer_id].trust_score if peer_id in self.peers else 0.5
            weighted[value] = weighted.get(value, 0) + weight
            total_weight += weight

        winner = max(weighted, key=weighted.get)
        confidence = weighted[winner] / total_weight if total_weight > 0 else 0

        if verbose:
            print(f"  Votes: {votes}")
            print(f"  Weights: ", end="")
            for pid in votes:
                w = self.peers[pid].trust_score if pid in self.peers else 0.5
                print(f"{pid}={w:.2f}  ", end="")
            print()
            print(f"  Winner: {winner} (confidence: {confidence:.2f})")

        return winner, confidence


print("ReputationNode defined. Ready for simulation.")

### 4A: Honest peers vs a liar

Setup: 4 peers observe the same events. 3 are honest (95% accurate).
1 is a liar (only 50% accurate -- reports random counts half the time).

Over 20 rounds, watch the liar's trust score drop.

In [ ]:
# 4A: Trust scores over time

random.seed(42)

node = ReputationNode("monitor")
profiles = {
    "honest-A": {"accuracy": 0.95, "uptime": 0.95},
    "honest-B": {"accuracy": 0.90, "uptime": 0.90},
    "honest-C": {"accuracy": 0.95, "uptime": 1.00},
    "liar":     {"accuracy": 0.50, "uptime": 0.80},
}

for pid in profiles:
    node.add_peer(pid)

print("Trust scores over 20 rounds:\n")
print(f"  {'Round':>5}  ", end="")
for pid in profiles:
    print(f"  {pid:>10}", end="")
print()
print(f"  {'-'*55}")

for rnd in range(1, 21):
    true_count = random.randint(100, 200)

    # Each peer reports
    reports = {}
    for pid, cfg in profiles.items():
        if random.random() < cfg["accuracy"]:
            reports[pid] = true_count  # accurate
        else:
            reports[pid] = true_count + random.randint(-50, 50)  # inaccurate

    # Determine majority (simple majority by count value)
    from collections import Counter
    majority_val = Counter(reports.values()).most_common(1)[0][0]

    # Record accuracy
    for pid, reported in reports.items():
        node.record_report(pid, was_accurate=(reported == majority_val))

    # Record heartbeat
    for pid, cfg in profiles.items():
        node.record_heartbeat(pid, responded=(random.random() < cfg["uptime"]))

    # Record contributions (all contribute equally for this sim)
    for pid in profiles:
        node.record_contribution(pid, 1)

    # Update scores
    node.update_all_scores()

    # Print
    if rnd % 5 == 0 or rnd == 1:
        print(f"  {rnd:>5}  ", end="")
        for pid in profiles:
            score = node.peers[pid].trust_score
            print(f"  {score:>10.3f}", end="")
        print()

# Final ranking
print(f"\nFinal ranking:")
ranked = sorted(node.peers.values(), key=lambda r: r.trust_score, reverse=True)
for r in ranked:
    bar = "#" * int(r.trust_score * 30)
    print(f"  {r.node_id:<12} trust={r.trust_score:.3f}  {bar}")

> **Question 10:** How many rounds does it take before the liar's trust score
> is clearly lower than the honest peers?

> **Question 11:** At round 1, everyone starts at 0.5 (neutral). Why?
> What would happen if new peers started at 1.0?

### 4B: Weighted voting in action

Now use the trust scores as voting weights. When the liar reports a
different count, does the majority-with-weights override them?

In [ ]:
# 4B: Weighted majority vote

print("=== Three audit scenarios ===\n")

# Scenario 1: All agree
votes1 = {"honest-A": 150, "honest-B": 150, "honest-C": 150, "liar": 150}
print("Scenario 1: Everyone agrees")
w1, c1 = node.weighted_majority_vote(votes1, verbose=True)

# Scenario 2: Liar disagrees
votes2 = {"honest-A": 150, "honest-B": 150, "honest-C": 150, "liar": 999}
print("\nScenario 2: Liar reports 999 (everyone else says 150)")
w2, c2 = node.weighted_majority_vote(votes2, verbose=True)

# Scenario 3: Two honest peers disagree
votes3 = {"honest-A": 150, "honest-B": 148, "honest-C": 150, "liar": 999}
print("\nScenario 3: One honest peer slightly off, liar reports 999")
w3, c3 = node.weighted_majority_vote(votes3, verbose=True)

> **Question 12:** In Scenario 2, the liar's vote carries less weight
> because of their low trust score. What would happen if 2 out of 4
> peers were liars? Could they outvote the honest peers?

> **Question 13:** The weights are (accuracy=0.5, uptime=0.3, reciprocity=0.2).
> What happens if you change accuracy to 0.9 and the others to 0.05?
> When would you want to emphasize uptime over accuracy?

### 4C: Experiment with trust weights

Change the weights and re-run the simulation to see how it affects trust detection.

In [ ]:
# 4C: Try different weight configurations

configs = [
    ("Balanced (default)",  (0.5, 0.3, 0.2)),
    ("Accuracy-heavy",      (0.8, 0.1, 0.1)),
    ("Uptime-heavy",        (0.2, 0.7, 0.1)),
    ("Reciprocity-heavy",   (0.1, 0.1, 0.8)),
]

print(f"{'Config':<25} {'honest-A':>10} {'honest-B':>10} {'honest-C':>10} {'liar':>10}")
print("-" * 68)

for label, weights in configs:
    # Re-run simulation with same data but different weights
    random.seed(42)
    n = ReputationNode("monitor")
    for pid in profiles:
        n.add_peer(pid)

    for rnd in range(1, 21):
        true_count = random.randint(100, 200)
        reports = {}
        for pid, cfg in profiles.items():
            if random.random() < cfg["accuracy"]:
                reports[pid] = true_count
            else:
                reports[pid] = true_count + random.randint(-50, 50)

        majority_val = Counter(reports.values()).most_common(1)[0][0]
        for pid, reported in reports.items():
            n.record_report(pid, was_accurate=(reported == majority_val))
        for pid, cfg in profiles.items():
            n.record_heartbeat(pid, responded=(random.random() < cfg["uptime"]))
        for pid in profiles:
            n.record_contribution(pid, 1)

        n.update_all_scores(weights)

    scores = {pid: n.peers[pid].trust_score for pid in profiles}
    print(f"{label:<25}", end="")
    for pid in profiles:
        print(f" {scores[pid]:>10.3f}", end="")
    print()

---
# Summary

You have explored the four algorithms that make a P2P network work:

| Algorithm | What you learned |
|-----------|-----------------|
| **Gossip** | Converges in O(log N) rounds. Random is good enough. |
| **Heartbeat** | Threshold tuning is a tradeoff. SUSPECT state prevents false positives. |
| **Choking** | Free-riders get punished. Optimistic unchoke solves cold start. |
| **Reputation** | Liars get detected through weighted voting. New peers start neutral. |

**Next:** You will wire these algorithms into your own P2P node (in any language)
that communicates over SQS with your classmates' nodes. The bots (`bot-alpha`,
`bot-bravo`, `bot-charlie`) will be running on the instructor's machine.

**Remember:** `bot-charlie` has 50% accuracy. Your reputation system should
detect this and downweight its votes in audits.

> Your node does not need to be in Python. It just needs to speak the
> JSON protocol over SQS. See the Protocol Reference for the full spec.